# Assignment 04

Caroline O’Sullivan (Boston University)  
September 27, 2025

In [3]:
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np

np.random.seed(45)

pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
#print("---This is Diagnostic check, No need to print it in the final doc---")

# df.printSchema() # comment this line when rendering the submission
#df.show(5)

#Missing Value Treatment
1. Replace missing values in Salary by Median of Salary based on the Employment Type, if missing then replace with the overall median of Salary

In [ ]:
#Missing Value Treatment
#1. Replace missing values in Salary by Median of Salary based on the Employment Type, if missing then replace with the overall median of Salary

from pyspark.sql import Window
from pyspark.sql.functions import col, when, isnan, count, lit, expr, avg, median
